In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

In [44]:
(x, y), (x_test, y_test) = datasets.fashion_mnist.load_data()
print(x.shape, y.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [45]:
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [46]:
batchsz = 128
db = tf.data.Dataset.from_tensor_slices((x, y))
db = db.map(preprocess).shuffle(10000).batch(batchsz)
db_iter = iter(db)
sample = next(db_iter)
print('batch: ', sample[0].shape, sample[1].shape)

batch:  (128, 28, 28) (128,)


In [47]:
db_test = tf.data.Dataset.from_tensor_slices((x, y))
db_test = db_test.map(preprocess).batch(batchsz)
db_test_iter = iter(db_test)
sample_test = next(db_test_iter)
print('batch_test: ', sample_test[0].shape, sample_test[1].shape)

batch_test:  (128, 28, 28) (128,)


In [48]:
model = Sequential([
    layers.Dense(256, activation=tf.nn.relu),  # [b, 784] => [b, 256]
    layers.Dense(128, activation=tf.nn.relu),  # [b, 256] => [b, 128]
    layers.Dense(64, activation=tf.nn.relu),  #  [b, 128] => [b, 64]
    layers.Dense(32, activation=tf.nn.relu),  # [b, 64] => [b, 32]
    layers.Dense(10)  # [b, 32] => [b, 10]
])
model.build(input_shape=[None, 28*28])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             multiple                  200960    
_________________________________________________________________
dense_16 (Dense)             multiple                  32896     
_________________________________________________________________
dense_17 (Dense)             multiple                  8256      
_________________________________________________________________
dense_18 (Dense)             multiple                  2080      
_________________________________________________________________
dense_19 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [49]:
optimizer = optimizers.Adam(lr=1e-3)

for epoch in range(10):
    for step, (x, y) in enumerate(db):
        # x: [b, 28, 28] => [b, 784]
        # y: [b]
        x = tf.reshape(x, [-1, 28*28])
        with tf.GradientTape() as tape:
            # [b, 784] => [b, 10]
            logits = model(x)
            y_onehot = tf.one_hot(y, depth=10)
            # [b]
            loss_mse = tf.reduce_mean(tf.losses.MSE(y_onehot, logits))
            loss_ce = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss_ce = tf.reduce_mean(loss_ce)
        grads = tape.gradient(loss_ce, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if step % 100 == 0:
            print(epoch, step, 'loss: ', float(loss_ce), float(loss_mse))

    # test
    total_correct = 0
    total_num = 0
    for x, y in db_test:
        # x: [b, 28, 28] => [b, 784]
        # y: [b]
        x = tf.reshape(x, [-1, 28*28])
        # [b, 10]
        logits = model(x)
        # logits => prob, [b, 10]
        prob = tf.nn.softmax(logits, axis=1)
        # [b, 10] => [b]
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)
        # pred: [b]
        # y: [b]
        # correct: [b], True: equal, False: not equal
        correct = tf.equal(pred, y)
        correct = tf.reduce_sum(tf.cast(correct, dtype=tf.int32))

        total_correct += int(correct)
        total_num += x.shape[0]
    acc = total_correct / total_num
    print(epoch, 'test acc:', acc)

        

0 0 loss:  2.310938835144043 0.18268868327140808
0 100 loss:  0.5975595712661743 14.885978698730469
0 200 loss:  0.5041289329528809 18.565391540527344
0 300 loss:  0.44202208518981934 17.49126434326172
0 400 loss:  0.4021662175655365 18.954387664794922
0 test acc: 0.8466166666666667
1 0 loss:  0.40464216470718384 21.012130737304688
1 100 loss:  0.3353254497051239 21.251258850097656
1 200 loss:  0.3543059825897217 25.930479049682617
1 300 loss:  0.5011817812919617 26.127384185791016
1 400 loss:  0.42418602108955383 21.295034408569336
1 test acc: 0.8727833333333334
2 0 loss:  0.28145715594291687 18.923219680786133
2 100 loss:  0.4275633692741394 27.821392059326172
2 200 loss:  0.40098434686660767 26.566009521484375


KeyboardInterrupt: 